In [1]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from soynlp.noun import LRNounExtractor, LRNounExtractor_v2
import re
from collections import Counter
import pandas as pd

In [2]:
# 데이터 불러오기
data = pd.read_csv('./youtube_channel_comments_data_20240606_104600.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56453 entries, 0 to 56452
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channelId           56453 non-null  object
 1   channelTitle        56453 non-null  object
 2   channelPublishedAt  56453 non-null  object
 3   subscriberCount     56453 non-null  int64 
 4   videoId             56453 non-null  object
 5   videoTitle          56453 non-null  object
 6   videoAuthorId       56453 non-null  object
 7   videoPublishedAt    56453 non-null  object
 8   duration            56453 non-null  object
 9   viewCount           56453 non-null  int64 
 10  likeCount           56453 non-null  int64 
 11  dislikeCount        56453 non-null  int64 
 12  commentId           56453 non-null  object
 13  commentAuthor       56325 non-null  object
 14  authorId            56453 non-null  object
 15  commentText         56449 non-null  object
 16  commentLikeCount    56

In [4]:
# cleaned_commentText의 데이터 보기
data['commentText']


0                         야채 많이드시고 가끔 요구르트 챙겨드시면\n어느정도 예방됨
1                            제일 확실한건 변비 설사 반복 됩니다 꼭 검사해보셔요
2                                           좋은정보 감사합니다 😊😊😊
3                                        걸리면 하느님 만나로 올라갈게요
4                                이중에 하나도 해당 안됐는데\n대장암 4기..
                               ...                        
56448            이지약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요\n너무 보고 싶었어요
56449                                     감사합니다 잘 지내셨죠~~!!
56450    이지 약사님, 정말 오랜만에 뵙네요. 그동안 잘 지내셨나요?\n너무 보고 싶었어요. ^^
56451            안녕하세요 감사합니다..ㅎㅎ 부활해 보았습니당..ㅎㅎ 무더위 잘 보내셨죠!
56452                 @@ezyaksa 이지 약사님을 이렇게 뵈니 너무 반갑네요. ㅎㅎ
Name: commentText, Length: 56453, dtype: object

In [5]:
# 특수 문자 제거 함수
def clean_text(text):
    # 줄바꿈 문자 포함하여 특수 문자 제거
    text = re.sub(r'[^ㄱ-ㅎ가-힣0-9\s]', '', text)
    # "ㅎㅎ", "ㅋㅋ" 등 비정식 단어 제거
    text = re.sub(r'ㅎㅎ|ㅋㅋ', '', text)
    text = re.sub(r'\s+', ' ', text).strip()  # 공백 문자를 하나의 공백으로 치환하고 양쪽 공백 제거
    return text

In [6]:
# NaN 값 제거
data = data.dropna(subset=['commentText'])

# NaN 값 공백 문자열로 대체
data['commentText'] = data['commentText'].fillna('')

# 댓글, 답글 데이터 전처리
data['cleaned_commentText'] = data['commentText'].apply(clean_text)

In [7]:
data['cleaned_commentText']

0                  야채 많이드시고 가끔 요구르트 챙겨드시면 어느정도 예방됨
1                    제일 확실한건 변비 설사 반복 됩니다 꼭 검사해보셔요
2                                       좋은정보 감사합니다
3                                걸리면 하느님 만나로 올라갈게요
4                           이중에 하나도 해당 안됐는데 대장암 4기
                           ...                    
56448     이지약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요 너무 보고 싶었어요
56449                                 감사합니다 잘 지내셨죠
56450    이지 약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요 너무 보고 싶었어요
56451             안녕하세요 감사합니다 부활해 보았습니당 무더위 잘 보내셨죠
56452                       이지 약사님을 이렇게 뵈니 너무 반갑네요
Name: cleaned_commentText, Length: 56449, dtype: object

In [8]:
# soynlp 훈련
word_extractor = WordExtractor()
word_extractor.train(data['cleaned_commentText'])
word_score_table = word_extractor.extract()

training was done. used memory 0.441 Gbory 0.227 Gb
all cohesion probabilities was computed. # words = 46279
all branching entropies was computed # words = 63382
all accessor variety was computed # words = 63382


### 1. soynlp 토크나이저 구축
soynlp의 WordExtractor로부터 얻은 word_score_table을 이용하여 LTokenizer 또는 MaxScoreTokenizer를 구성할 수 있습니다.

In [9]:
# 코헤전스 점수 계산
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
# 명사 추출
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(data['cleaned_commentText']) # list of str like

noun_scores = {noun:score.score for noun, score in nouns.items()}
combined_scores = {noun:score + scores.get(noun, 0)
    for noun, score in noun_scores.items()}

combined_scores.update(
    {subword:cohesion for subword, cohesion in scores.items()
    if not (subword in combined_scores)}
)

tokenizer = LTokenizer(scores=combined_scores)
# maxscore_tokenizer = MaxScoreTokenizer(scores=combined_scores)



[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 145468 from 56449 sents. mem=0.453 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=665231, mem=0.682 Gb
[Noun Extractor] batch prediction was completed for 43450 words
[Noun Extractor] checked compounds. discovered 28041 compounds
[Noun Extractor] postprocessing detaching_features : 38152 -> 29037
[Noun Extractor] postprocessing ignore_features : 29037 -> 28831
[Noun Extractor] postprocessing ignore_NJ : 28831 -> 28390
[Noun Extractor] 28390 nouns (28041 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.754 Gb                    
[Noun Extractor] 69.04 % eojeols are covered


### 2. 텍스트 토크나이징
토크나이저를 사용하여 텍스트 데이터를 토큰화합니다.



In [10]:
# cohension score인 scores dict 변수에 직접 추가
combined_scores["네츄럴팩터스"] = 1.0
# scores["레게노"] = 1.0
# scores["돈쭐"] = 1.0



data['tokenized'] = data['cleaned_commentText'].apply(tokenizer.tokenize)
# data['maxscore_tokenized'] = data['cleaned_commentText'].apply(maxscore_tokenizer.tokenize)

In [11]:
# LTokenizer
# 한글자, 지시대명사 등 제거
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if len(item) > 1])

# 한글자 이외의 형용사, 부사, 동사 불용어 stopword 목록 작성하여 반영
stopword = ['ㅎㅎ', 'ㅋㅋ', 'ㅠㅠ', 'ㅜㅜ', 'ㅇㅇ', 'ㅈㅈ', 'ㅊㅊ', 'ㅋㅋ', 'ㅎㅎ', 'ㅇㅇ', 'ㅈㅈ', 'ㅊㅊ']
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopword])


In [12]:
# 각 행에 대해 l_tokenized와 maxscore_tokenized 결과를 함께 출력
for index, row in data.iterrows():
    print(f"Row {index}:")
    print("L Tokenized: ", row['tokenized'])
    # print("MaxScore Tokenized: ", row['maxscore_tokenized'])
    # print("\n")
    if index > 100:
        break


Row 0:
L Tokenized:  ['야채', '많이', '드시고', '가끔', '요구르트', '챙겨', '드시면', '어느정도', '예방']
Row 1:
L Tokenized:  ['제일', '확실', '한건', '변비', '설사', '반복', '됩니', '검사', '해보셔요']
Row 2:
L Tokenized:  ['좋은정보', '감사', '합니다']
Row 3:
L Tokenized:  ['걸리', '하느님', '만나로', '올라', '갈게요']
Row 4:
L Tokenized:  ['이중', '하나', '해당', '안됐는데', '대장암', '4기']
Row 5:
L Tokenized:  ['힘내세요']
Row 6:
L Tokenized:  []
Row 7:
L Tokenized:  ['헉진짜', '힘내세요', '좋은일', '가득', '하시길바래요']
Row 8:
L Tokenized:  ['힘내세요']
Row 9:
L Tokenized:  ['체중감소', '에서', '안심']
Row 10:
L Tokenized:  ['질병', '없는', '사람', '먹어도', '되나요']
Row 11:
L Tokenized:  ['내츄럴', '팩터스', '베르베린', '먹고있는데', '휴지기', '가져야', '할까요']
Row 12:
L Tokenized:  ['제일', '마지막', '제품', '내츄럴팩터', '베르베린', '리포미셀', '구매', '했는데', '하루', '몇번', '몇알', '섭취', '하는', '건가요']
Row 13:
L Tokenized:  ['당뇨환자', '먹고', '있는데', '먹을', '있는', '영양제', '알려주세요']
Row 14:
L Tokenized:  ['샘요즘', '알코틴클렌저', '라고', '니코틴', '알코올', '분해', '해준다는', '영양제', '광고', '해서', '약국', '에서도', '판매', '한다고', '되어있는데', '효과가', '정말', '있는', '건가요']
Row 15:
L Tokenized:  

In [14]:
# 채널별로 데이터 그룹화 및 토큰 리스트 합치기
grouped_data = data.groupby('channelId')['tokenized'].agg(lambda x: [token for sublist in x for token in sublist])
grouped_data


channelId
UC3iSLVH0MxHfwO69oHKpvog    [리틀약사, 최고이십니다, 감사, 합니다, 비문증, 브로멜라인, 파파인에, 대해서,...
UC6ggXTuBVchhwHeQ12Ita1w    [베르베린, 복용, 방법, 안내, 복용, 이미, 아시겠지만, 점진적으로, 증량, 하...
UCCMFTDGarjgZLc1DlIbbvRg    [잘보고, 갑니다, 감사, 합니다, 사람, 소화, 시켜, 열량을, 뽑아, 없는, 셀...
UCMFk5S7g5DY-CZNVh_Kyz_A    [야채, 많이, 드시고, 가끔, 요구르트, 챙겨, 드시면, 어느정도, 예방, 제일,...
UCY-mXLM6DsS9cmSwlh0tqSA    [담백하루, 아르기닌, 변비, 유산균, 공구, 오픈, 구매, 링크, 트랜짓, 유산균...
Name: tokenized, dtype: object

In [15]:
# 각 채널별로 상위 20개 단어 추출
top_words_per_channel = {}
for channel_id, tokens in grouped_data.items():
    # 토큰의 빈도수 계산
    token_counts = Counter(tokens)
    # 가장 빈번한 30개 단어 추출
    top_words = token_counts.most_common(30)
    top_words_per_channel[channel_id] = top_words

In [16]:
# 결과 출력
for channel_id, words in top_words_per_channel.items():
    print(f"Channel {channel_id}:")
    for word, count in words:
        print(f"{word}: {count}")
    print("\n")

Channel UC3iSLVH0MxHfwO69oHKpvog:
합니다: 1531
감사: 1276
으로: 830
상담: 615
마그네슘: 553
비타민: 548
먹고: 543
하고: 541
리틀약사: 525
영양제: 517
밴드: 508
제품: 488
복용: 465
영상: 463
많이: 426
약사님: 398
네이버: 398
좋은: 378
83355090: 372
오메가3: 367
너무: 365
입니다: 365
하시면: 354
건강: 353
보다: 350
해서: 346
하는: 346
섭취: 342
먹어도: 341
도움: 340


Channel UC6ggXTuBVchhwHeQ12Ita1w:
합니다: 749
감사: 676
복용: 631
약사님: 485
영상: 312
으로: 273
섭취: 251
좋은: 251
영양제: 230
하고: 224
효과: 216
제품: 215
있습니다: 212
에서: 203
정보: 193
하세요: 177
선생님: 176
입니다: 168
하는: 165
많이: 165
너무: 163
건강: 160
저는: 160
도움: 158
멜라토: 158
먹고: 154
레스베라트롤: 153
베르베린: 151
피세틴: 151
안녕: 145


Channel UCCMFTDGarjgZLc1DlIbbvRg:
감사: 84
합니다: 72
약사님: 36
이지약사님: 33
효과: 32
너무: 30
좋은: 29
제품: 24
멜라토: 24
저도: 23
같이: 22
바르고: 20
피부: 19
입니다: 19
영상: 19
정보: 18
해요: 18
크림: 17
으로: 15
저는: 14
에서: 14
있는데: 14
하는: 13
이지: 13
처방: 13
오랜만: 13
하세요: 13
있어요: 13
멜라노사: 13
궁금: 12


Channel UCMFk5S7g5DY-CZNVh_Kyz_A:
합니다: 10553
감사: 8302
섭취: 5436
영양제: 3379
비타민: 3110
약사님: 3055
으로: 3051
제품: 2882
오메가3: 2878
복용: 2815
영상: 2587
드립니다: 2579